In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_US_df = pd.read_excel(io='###.xlsx',sheet_name='US')
file_International_df = pd.read_excel(io='###.xlsx',sheet_name='Intl')
us_df = file_US_df.copy()
in_df = file_International_df.copy()

In [ ]:
us_df.head()

In [ ]:
us_df = us_df.fillna(0)

In [ ]:
us_df['Year']=us_df['Date'].dt.year #adding a year and month column to ease grouping
us_df['Month']=us_df['Date'].dt.month
us_df.head()

In [ ]:
#print(us_df.loc[(us_df['Year']==2017)&(us_df['Month']==9)]['Date'].unique())

In [ ]:
#grouping by Year for comparisson with yearly totals
us_by_year = us_df[us_df['Year'].isin([2017,2018,2019])].groupby(['Campus','Year'],as_index=False)['Total'].sum()
pd.set_option("display.max_rows",1000)
us_by_year.head()

In [ ]:
us_total = us_by_year.groupby(['Year'],as_index=False).sum()
us_total

In [ ]:
total_pivoted = us_by_year.copy().pivot_table(values=['Total'],index=['Year'],aggfunc=np.sum,fill_value=0,margins=True)
total_pivoted

In [ ]:
#let's now try to figure out the monthly averages2017-2019
us_by_month = us_df[us_df['Year'].isin([2017,2018,2019])].groupby(['Campus','Year','Month'],as_index=False).agg({'Total':np.sum,'Date':'nunique'})
us_by_month = us_by_month.rename({'Date':'Weeks'},axis=1)
us_by_month['Month_Avg'] = (us_by_month['Total']/us_by_month['Weeks']).astype('int32')
us_by_month

In [ ]:
us_pivoted_month = pd.pivot_table(us_by_month,values=['Total','Month_Avg'],index=['Year','Month'],aggfunc={'Total':np.sum,'Month_Avg':np.sum}).round(0).astype('int32')
us_pivoted_month

In [ ]:
#with the previous method we got an error of +- 5 or +- 0.02. I'll do a avg of the total and see

us_by_month_all = us_df[us_df['Year'].isin([2017,2018,2019])].groupby(['Year','Month'],as_index=True).agg({'Total':np.sum,'Date':'nunique'})
us_by_month_all = us_by_month_all.rename({'Date':'Weeks'},axis=1)
us_by_month_all['Avg'] = (us_by_month_all['Total']/us_by_month_all['Weeks']).round(0).astype('int32')
us_by_month_all

## Conclusion

this last method gives us exactly the number in Roambi. Let's now apply that to international locations too

## Calculating monthly averages for Internationals

In [ ]:
in_df = in_df.fillna(0)
in_df.head()

In [ ]:
in_df['Year'] = in_df['Weekend'].dt.year
in_df['Month'] = in_df['Weekend'].dt.month
in_df.head()

In [ ]:
in_by_year = in_df[in_df['Year'].isin([2017,2018,2019])].groupby(['Campus','Year'],as_index=False)['Total'].sum()
in_total = in_by_year.groupby(['Year'],as_index=False).sum()
in_total

In [ ]:
in_by_month = in_df[in_df['Year'].isin([2017,2018,2019])].groupby(['Year','Month'],as_index=True).agg({'Total':np.sum,'Weekend':'nunique'}).rename({'Weekend':'Weeks'},axis=1)
in_by_month['Avg'] = (in_by_month['Total']/in_by_month['Weeks']).round(0).astype('int32')
in_by_month

# Conclusion 
I have now the correct results from the US and Intl tabs on the CSV file compared to the Roambi. these two dataframes, **us_by_month_all and in_by_month**, will be from now on my reference to compare when comparing the factmetrics table and the calculations from the Warehouse

In [ ]:
writer = pd.ExcelWriter('month averages from csv.xlsx',engine='xlsxwriter')

In [ ]:
us_by_month_all.to_excel(writer,sheet_name='US')
in_by_month.to_excel(writer,sheet_name='Intl')

In [ ]:
in_df['Service'].value_counts()

In [ ]:
inVolunteers_by_month = in_df[(in_df['Year'].isin([2017,2018,2019]))&(in_df['Service'].str.contains('Volunteers',na=False))].groupby(['Year','Month'],as_index=False).agg({'Total':np.sum,'Weekend':'nunique'}).rename({'Weekend':'Weeks'},axis=1)
inVolunteers_by_month['Avg'] = (inVolunteers_by_month['Total']/inVolunteers_by_month['Weeks']).round(0).astype('int32')
inVolunteers_by_month

In [ ]:
inVolunteers_by_month.to_excel(writer,sheet_name='Volunteers')
writer.save()